# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import sys
sys.path.append("..")
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,derbent,42.0678,48.2899,298.09,63,0,3.38,RU,1722454984
1,1,"poblacion, san felipe",14.5665,121.0308,301.13,91,40,0.51,PH,1722454984
2,2,awjilah,29.1081,21.2869,303.54,40,0,6.83,LY,1722454757
3,3,kodiak,57.7900,-152.4072,290.88,45,0,9.26,US,1722454985
4,4,puerto natales,-51.7236,-72.4875,275.40,93,75,2.06,CL,1722454985


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

coordinates = {"City": city_data_df["City"],
               "latitude": city_data_df["Lat"],
               "longitude": city_data_df["Lng"],
               "Humidity": city_data_df["Humidity"]}


coordinates_df = pd.DataFrame(coordinates)

# Configure the map_plot
map_plot = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,derbent,42.0678,48.2899,298.09,63,0,3.38,RU,1722454984
1,1,"poblacion, san felipe",14.5665,121.0308,301.13,91,40,0.51,PH,1722454984
2,2,awjilah,29.1081,21.2869,303.54,40,0,6.83,LY,1722454757
3,3,kodiak,57.7900,-152.4072,290.88,45,0,9.26,US,1722454985
4,4,puerto natales,-51.7236,-72.4875,275.40,93,75,2.06,CL,1722454985
...,...,...,...,...,...,...,...,...,...,...
586,586,aripuana,-9.1667,-60.6333,310.10,19,1,1.68,BR,1722456190
587,587,isyangulovo,52.1909,52.1909,292.96,51,100,2.76,RU,1722456191
588,588,zima,53.9202,102.0442,289.54,97,100,1.59,RU,1722456192
589,589,khanty-mansiysk,61.0042,69.0019,290.21,94,0,2.00,RU,1722456194


In [5]:
city_data_df["Humidity"] <= 50

0      False
1      False
2       True
3       True
4      False
       ...  
586     True
587    False
588    False
589    False
590    False
Name: Humidity, Length: 591, dtype: bool

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.dropna()

city_filter_df = city_data_df.loc[(city_data_df["Cloudiness"] <50)\
& (city_data_df["Humidity"] <= 80)\
  & (city_data_df["Max Temp"] <= 290)\
  &(city_data_df["Max Temp"] >= 270)] 

# Display sample data
city_filter_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,ushuaia,-54.8000,-68.3000,277.96,60,40,5.14,AR,1722455185
12,12,edinburgh of the seven seas,-37.0676,-12.3116,284.73,55,21,7.12,SH,1722455188
14,14,ulsteinvik,62.3432,5.8487,287.65,75,11,3.58,NO,1722455190
15,15,ghanzi,-21.5667,21.7833,289.29,18,3,1.67,BW,1722455192
17,17,qaqortoq,60.7167,-46.0333,286.64,41,0,4.82,GL,1722455194
76,76,tsiombe,-25.3000,45.4833,289.43,75,0,0.68,MG,1722455063
78,78,iqaluit,63.7506,-68.5145,283.00,66,20,4.63,CA,1722455277
90,90,port alfred,-33.5906,26.8910,285.30,73,0,6.72,ZA,1722455293
119,119,alesund,62.4723,6.1549,287.66,72,40,3.60,NO,1722455154
133,133,longyearbyen,78.2186,15.6401,285.06,76,0,5.14,SJ,1722455352


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_filter_df[["City",
                           "Country",
                           "Lat",
                           "Lng",
                           "Humidity"]].copy() 

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,ushuaia,AR,-54.8000,-68.3000,60,
12,edinburgh of the seven seas,SH,-37.0676,-12.3116,55,
14,ulsteinvik,NO,62.3432,5.8487,75,
15,ghanzi,BW,-21.5667,21.7833,18,
17,qaqortoq,GL,60.7167,-46.0333,41,
76,tsiombe,MG,-25.3000,45.4833,75,
78,iqaluit,CA,63.7506,-68.5145,66,
90,port alfred,ZA,-33.5906,26.8910,73,
119,alesund,NO,62.4723,6.1549,72,
133,longyearbyen,SJ,78.2186,15.6401,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
response = requests.get(base_url, params=params)

NameError: name 'base_url' is not defined

In [ ]:
geoapify_key

'403892d05e5a475d8d2869c2f43253ef'

In [ ]:
base_url

'https://api.geoapify.com/v2/places'

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)


    name_address = response.json()
  
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
# hotel_df


Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
edinburgh of the seven seas - nearest hotel: No hotel found
ulsteinvik - nearest hotel: Quality Hotel Ulstein
ghanzi - nearest hotel: Ghanzi farmhouse
qaqortoq - nearest hotel: Hotel Qaqortoq
tsiombe - nearest hotel: No hotel found
iqaluit - nearest hotel: Frobisher Inn
port alfred - nearest hotel: No hotel found
alesund - nearest hotel: Brosundet
longyearbyen - nearest hotel: The Vault
potosi - nearest hotel: Hotel Tarija
linxia chengguanzhen - nearest hotel: 如家酒店
springbok - nearest hotel: Elkoweru Guesthouse
plettenberg bay - nearest hotel: Milkwood Manor
hermanus - nearest hotel: Aloe guest house
nuuk - nearest hotel: Nordbo
rikaze - nearest hotel: Gesar Hotel
port saint john's - nearest hotel: Outback Inn
de aar - nearest hotel: No hotel found
menongue - nearest hotel: No hotel found
saldanha - nearest hotel: Hoedjesbaai
east london - nearest hotel: No hotel found
usinsk - nearest hotel: общежитие Лукойл-К

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

%%capture --no-display

coordinates = {"City": city_data_df["City"],
               "latitude": city_data_df["Lat"],
               "longitude": city_data_df["Lng"],
               "Humidity": city_data_df["Humidity"]}


coordinates_df = pd.DataFrame(coordinates)

# Configure the map_plot
map_plot = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)